## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head(3)

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [7]:
# Convert your X data to numeric data types however you see fit
# Convert column "OverTime" to Numeric
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

# Then transform to 0/1
X_train_encoded["OverTime"] = X_train_encoded["OverTime"].map({'No': 0, 'Yes': 1}) 
X_test_encoded["OverTime"] = X_test_encoded["OverTime"].map({'No': 0, 'Yes': 1}) 

In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded)

# Scale the training and testing data
X_train_scaled =scaler.transform(X_train_encoded)
X_test_scaled =scaler.transform(X_test_encoded)


In [9]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoder = dept_encoder.transform(y_train[['Department']])
y_test_dept_encoder = dept_encoder.transform(y_test[['Department']])

In [10]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_encoder = attr_encoder.transform(y_train[['Attrition']])
y_test_attr_encoder = attr_encoder.transform(y_test[['Attrition']])

## Part 2: Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data.
num_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_features,))

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu')(input_layer)
shared2 = layers.Dense(128, activation='relu')(shared1)


In [12]:
# Create a branch for Department
# with a hidden layer and an output layer
# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_hidden = layers.Dense(32, activation='relu')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[attrition_output, department_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics={
        'attrition_output': 'accuracy',
        'department_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      8,320 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
history = model.fit(
    X_train_scaled,
    [y_train_attr_encoder, y_train_dept_encoder],
    epochs=100,
    batch_size=32,
    verbose=1
)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - attrition_output_accuracy: 0.7554 - attrition_output_loss: 0.5621 - department_output_accuracy: 0.5240 - department_output_loss: 0.9591 - loss: 1.5210
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8388 - attrition_output_loss: 0.4158 - department_output_accuracy: 0.6659 - department_output_loss: 0.7766 - loss: 1.1923 
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8420 - attrition_output_loss: 0.3953 - department_output_accuracy: 0.6660 - department_output_loss: 0.7675 - loss: 1.1629 
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8651 - attrition_output_loss: 0.3536 - department_output_accuracy: 0.6738 - department_output_loss: 0.7419 - loss: 1.0952 
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8636 - attrition_output_loss: 0.3146 - department_output_accuracy: 0.6401 - department_output_loss:

In [16]:
# Evaluate the model with the testing data
test_results = model.evaluate(
    X_test_scaled,
    [y_test_attr_encoder, y_test_dept_encoder],
    verbose = 1)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8430 - attrition_output_loss: 1.7601 - department_output_accuracy: 0.5416 - department_output_loss: 2.8989 - loss: 4.6572  


In [17]:
# Print the accuracy for both department and attrition
print(f'The Attrition accuracy of this model is: {test_results[1]}')
print(f'The Attrition accuracy of this model is: {test_results[2]}')

The Attrition accuracy of this model is: 1.4344924688339233
The Attrition accuracy of this model is: 2.7767410278320312


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric here  for this data - especially for Attrition Prediction.  The data is imballanced (more No than Yes responses), so it is possible to acheive high accuracy by simply predicting No for all.  Better Metrics might include: Precision & recall, F1 Scorings, ROC-AUC and or Confusion matrix analysis.
2. I selected Softmax because Both are Multi-class classification problems.  Softmax probabilities sum up to 1 so this was also a good choice for mutually exclusive classes as we have here.  Also since I was using categorical_crossentropy loss, it just makes good sense as Softmax works well well with this function.
3. This model could benefit from: Add more features, use class weights to better handle imbalanced data.  Use regularization techniques to help prevent overfitting. Hyperparameter tuning.  Appluying diffentert arcetectures could help also.  Finally, use cross-validation (instead of just using just one train/test split).
